In [1]:
import pandas as pd
import numpy as np
import keras
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import sklearn.metrics as metrics

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
healthy_dirs = [
    r'/content/drive/MyDrive/chest_xray/test/NORMAL',
    r'/content/drive/MyDrive/chest_xray/train/NORMAL',
    r'/content/drive/MyDrive/chest_xray/val/NORMAL',
]

pneumonia_dir = [
    r'/content/drive/MyDrive/chest_xray/test/PNEUMONIA',
    r'/content/drive/MyDrive/chest_xray/train/PNEUMONIA',
    r'/content/drive/MyDrive/chest_xray/val/PNEUMONIA'
]

filepaths = []
labels = []
dict_lists = [healthy_dirs, pneumonia_dir]
class_labels = ['Normal', 'Pneumonia']

for i, dir_list in enumerate(dict_lists):
    for j in dir_list:
        flist = os.listdir(j)
        for f in flist:
            fpath = os.path.join(j, f)
            filepaths.append(fpath)
            labels.append(class_labels[i])

Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
pneumonia_data = pd.concat([Fseries, Lseries], axis=1)
pneumonia_df = pd.DataFrame(pneumonia_data)
print(pneumonia_df.head())
print(pneumonia_df["labels"].value_counts())

                                           filepaths  labels
0  /content/drive/MyDrive/chest_xray/test/NORMAL/...  Normal
1  /content/drive/MyDrive/chest_xray/test/NORMAL/...  Normal
2  /content/drive/MyDrive/chest_xray/test/NORMAL/...  Normal
3  /content/drive/MyDrive/chest_xray/test/NORMAL/...  Normal
4  /content/drive/MyDrive/chest_xray/test/NORMAL/...  Normal
labels
Pneumonia    4280
Normal       1583
Name: count, dtype: int64


In [5]:
pneumonia_df.head()

,filepaths,labels
0,/content/drive/MyDrive/chest_xray/test/NORMAL/...,Normal
1,/content/drive/MyDrive/chest_xray/test/NORMAL/...,Normal
2,/content/drive/MyDrive/chest_xray/test/NORMAL/...,Normal
3,/content/drive/MyDrive/chest_xray/test/NORMAL/...,Normal
4,/content/drive/MyDrive/chest_xray/test/NORMAL/...,Normal


In [6]:
train_images, test_images = train_test_split(pneumonia_df, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(train_images, test_size=0.2, random_state=42)

In [7]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)

train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=4,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_images,x_col="filepaths", y_col="labels",
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=4,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(224,224),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=4,
                                    shuffle=False
                                   )

Found 3752 validated image filenames belonging to 2 classes.
Found 1173 validated image filenames belonging to 2 classes.
Found 938 validated image filenames belonging to 2 classes.


In [8]:
classes=list(train.class_indices.keys())
print (classes)

['Normal', 'Pneumonia']


In [9]:
pneumonia_df.to_csv('pneumonia_dataset.csv', index=False)

In [10]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [11]:
def create_model(model_name):
    if model_name == 'CheXpert':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'COVID-Net':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    else:
        raise ValueError("Invalid model name. Choose from: 'CheXpert', 'COVID-Net', 'ResNet', 'InceptionV3'")

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add a new classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2, activation='softmax')(x)  # Binary classification output with softmax

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [12]:
model=create_model('InceptionV3')

87910968/87910968 [==============================] - 0s 0us/step


In [13]:
from tensorflow.keras.applications.vgg16 import VGG16

def create_vgg16_model():
    # Load the pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add a new classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [14]:
vgg16_model = create_vgg16_model()

58889256/58889256 [==============================] - 0s 0us/step


In [19]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
history=vgg16_model.fit(train, epochs=10, validation_data=val, callbacks=[es])

Epoch 1/10
938/938 [==============================] - 73s 78ms/step - loss: 0.1514 - accuracy: 0.9464 - val_loss: 0.1635 - val_accuracy: 0.9424
Epoch 2/10
938/938 [==============================] - 64s 68ms/step - loss: 0.1461 - accuracy: 0.9464 - val_loss: 0.1598 - val_accuracy: 0.9424
Epoch 3/10
938/938 [==============================] - 64s 68ms/step - loss: 0.1385 - accuracy: 0.9488 - val_loss: 0.1630 - val_accuracy: 0.9392


In [21]:
y_pred = vgg16_model.predict(test)

294/294 [==============================] - 23s 79ms/step


In [17]:
pred = np.argmax(y_pred, axis=1) #pick class with highest  probability

labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2 = [labels[k] for k in pred]

In [18]:

from sklearn.metrics import confusion_matrix, accuracy_score

y_test = test_images.labels
print(classification_report(y_test, pred2))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2)*100))

              precision    recall  f1-score   support

      Normal       0.95      0.88      0.91       341
   Pneumonia       0.95      0.98      0.97       832

    accuracy                           0.95      1173
   macro avg       0.95      0.93      0.94      1173
weighted avg       0.95      0.95      0.95      1173

Accuracy of the Model: 95.1%


In [20]:
#inception
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
history=model.fit(train, epochs=10, validation_data=val, callbacks=[es])

Epoch 1/10
938/938 [==============================] - 89s 88ms/step - loss: 0.2709 - accuracy: 0.8899 - val_loss: 0.1972 - val_accuracy: 0.9200
Epoch 2/10
938/938 [==============================] - 69s 73ms/step - loss: 0.1947 - accuracy: 0.9256 - val_loss: 0.3522 - val_accuracy: 0.8827
Epoch 3/10
938/938 [==============================] - 67s 71ms/step - loss: 0.1624 - accuracy: 0.9363 - val_loss: 0.3758 - val_accuracy: 0.8678


In [22]:
y_pred_inception = model.predict(test)

294/294 [==============================] - 21s 66ms/step


In [23]:
pred_inception = np.argmax(y_pred_inception, axis=1) #pick class with highest  probability

labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_inception = [labels[k] for k in pred_inception]

In [24]:

from sklearn.metrics import confusion_matrix, accuracy_score

y_test = test_images.labels
print(classification_report(y_test, pred2_inception))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_inception)*100))

              precision    recall  f1-score   support

      Normal       0.89      0.93      0.91       341
   Pneumonia       0.97      0.95      0.96       832

    accuracy                           0.95      1173
   macro avg       0.93      0.94      0.94      1173
weighted avg       0.95      0.95      0.95      1173

Accuracy of the Model: 94.8%


In [25]:
ensemble_preds = (y_pred + y_pred_inception) / 2
ensemble_preds=np.argmax(ensemble_preds, axis=1)

In [26]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2_ensemble = [labels[k] for k in ensemble_preds]

In [27]:

from sklearn.metrics import confusion_matrix, accuracy_score

y_test = test_images.labels
print(classification_report(y_test, pred2_ensemble))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2_ensemble)*100))

              precision    recall  f1-score   support

      Normal       0.94      0.93      0.93       341
   Pneumonia       0.97      0.97      0.97       832

    accuracy                           0.96      1173
   macro avg       0.95      0.95      0.95      1173
weighted avg       0.96      0.96      0.96      1173

Accuracy of the Model: 96.2%


In [28]:
vgg16_model.save('/content/drive/MyDrive/chest_xray/chest_xray_vgg16.keras')

In [29]:
model.save('/content/drive/MyDrive/chest_xray/chest_xray_inception.keras')